In [1]:
import numpy as np
import cupy as qp
import pylab as pl
import numba as nb
import timeit
xp = qp

In [26]:
NX = 20000
NY = 10000
nx = 2000
ny = 2000
steps = 20
overX = 2*steps
overY = 2*steps
nx = 2000+overX
ny = 2000+overY
np.ceil(NY/(ny-overY))

5.0

In [27]:
A = np.arange((NX*NY),dtype=np.float32).reshape((NX,NY)) 
AMAX = A.copy()
AMIN = A.copy() # ((NX,NY))
H = A.copy()
UU = A.copy()
VV = A.copy()

In [28]:
xp = qp
#@nb.njit()
#@xp.fuse() # this fails because fuse doesn't like indexing.
def diffit(A,B,DX):
    B[:,0:-1] = (A[:,1:]-A[:,0:-1])/DX
    B[:,-1]= B.dtype.type(0.0)  # I may not need this in the patchwise
    #print (B.dtype,B.shape)
    return 


In [29]:
#@nb.njit()
def blockcomp(B,DX,NX=NX,NY=NY,nx=nx,ny=ny,overX=overX,overY=overY):
    A = xp.empty((nx,ny),dtype=xp.float32)
    C = xp.empty((nx,ny),dtype=A.dtype)
    for i in range(0, NX, nx-overX):
        for j in range(0, NY,ny-overY):
            tmp = B[i:i+nx,j:j+ny]  # subscripts past ends will truncate!
            U = tmp.shape           # therefore we need to see what shape really is
            A[:U[0],:U[1]] = xp.asarray(tmp)
            AP = A[:U[0],:U[1]]
            D = C[:U[0],:U[1]]
            for s in range(steps):
                 diffit(AP,D,DX) 
                 AP,D = D,AP
            tmp[:,:] = qp.asnumpy(AP) # put it back in same home
blockcomp(A,2)               

In [30]:
#     ' int j = _ind.size() ; z = 0*x + 0*y[_ind.size() - i - 1]+i ;',
add_reverse = qp.ElementwiseKernel(
     'T x, raw T y, int64 w, T DX', 'T z',
     ' z = (y[i]-y[i+1])/DX ;',
     'add_reverse')
t0 = qp.arange(3,dtype=qp.float32)
t1 = qp.arange(12,dtype=qp.float32).reshape((3,4))
t2 = add_reverse(t0,t1,3,3)
print(qp.asnumpy(t2))
print(qp.asnumpy(t1))

#@qp.fuse()
def blockcomp2(B,H,UU,VV,BMAX,BMIN,DX,kernelMethod=False,xp=qp, NX=NX,NY=NY,nx=nx,ny=ny,overX=overX,overY=overY):

    DX = qp.float32(DX)
    A = xp.empty((nx,ny),dtype=xp.float32)
    C = xp.empty((nx,ny),dtype=A.dtype)
    AMAX = xp.empty((nx,ny),dtype=xp.float32)
    AMIN = xp.empty((nx,ny),dtype=xp.float32)
    GU  = xp.empty((nx,ny),dtype=xp.float32)
    GV = xp.empty((nx,ny),dtype=xp.float32)
    GH = xp.empty((nx,ny),dtype=xp.float32)
    for i in range(0, NX, nx-overX):
        for j in range(0, NY,ny-overY):
            tmp = B[i:i+nx,j:j+ny]  # subscripts past ends will truncate!
            U = tmp.shape           # therefore we need to see what shape really is
            AP = A[:U[0],:U[1]] = xp.asarray(tmp)
            D = C[:U[0],:U[1]]
            
            amax = AMAX[0:U[0],0:U[1]] =  AP[:,:]  # copy it
            amin = AMIN[0:U[0],0:U[1]] =  AP[:,:]  # copy it
            
            gh = GH[0:U[0],0:U[1]] = xp.asarray(H[i:i+nx,j:j+ny] )
            
            tmpU =  xp.asarray(UU[i:i+nx,j:j+ny])
            US = tmpU.shape
            gu = GU[0:US[0],0:US[1]]= xp.asarray(tmpU)
            tmpV =  xp.asarray(UU[i:i+nx,j:j+ny])
            VS = tmpV.shape
            gv = GV[0:VS[0],0:VS[1]]= xp.asarray(tmpV)
            
            coastMAP = gh>20
            
            for s in range(steps):
                if kernelMethod:
                    D = add_reverse(AP,AP,U[1],DX)
                else:
                    diffit(AP,D,DX)
                AP,D = D,AP
                if i==0:
                    pass  # apply boundary condition on left
                if i+nx  >= NX:
                    pass # apply boundary conditions on right
                if j==0: 
                    pass # apply boundary condition on top
                if j+ny >= NY:
                    pass # apply boundary conditions on bottom
                
                # apply coastal boundary conditions
                # gv *= coastMap
                # gu *= coastMap
                
                amax[:,:] = xp.maximum(AP,amax)
                amin[:,:] = xp.minimum(AP,amin)
            #now we need to move this back from device.
            #tricky part:
            # want to discard or overwrite the invalid data in step-width border along edge
            # However we don't do that along the border edges.
            # THe following isnt' doing this right:
            tmp[:,:] = qp.asnumpy(AP) # put it back in same home
            BMAX[i:i+nx,j:j+ny] = np.maximum( qp.asnumpy(amax),BMAX[i:i+nx,j:j+ny])
            BMIN[i:i+nx,j:j+ny] = np.minimum( qp.asnumpy(amin), BMIN[i:i+nx,j:j+ny])
    del A,C,AMAX,AMIN,DX
    del AP,D,amax,amin  # are these out of scope anyhow?
        
blockcomp2(A,H,UU,VV,AMAX,AMIN,2)       

[-0.33333334 -0.33333334 -0.33333334]
[[ 0.  1.  2.  3.]
 [ 4.  5.  6.  7.]
 [ 8.  9. 10. 11.]]


In [31]:
steps

20

In [32]:
%timeit blockcomp2(A,H,UU,VV,AMAX,AMIN,2,kernelMethod=True,ny=int(ny*5))

2.59 s ± 16 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:


%timeit blockcomp2(A,H,UU,VV,AMAX,AMIN,2,kernelMethod=False)

1.1 s ± 2.13 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:

%timeit blockcomp2(A,H,UU,VV,AMAX,AMIN,2,kernelMethod=False,xp=np)

1.49 s ± 5.27 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [33]:
%timeit diffit(A,AMAX,2)

378 ms ± 251 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
add_reverse = qp.ElementwiseKernel(
     'T x, raw T y, int64 w', 'T z',
     ' z = y[i]-y[i+w] ;',
     'add_reverse')
t0 = qp.arange(3,dtype=qp.float32)
t1 = qp.arange(12,dtype=qp.float32).reshape((3,4))
t2 = add_reverse(t1,t1,4)
print(qp.asnumpy(t2))
print(qp.asnumpy(t1))

[[-4.        -4.        -4.        -4.       ]
 [-4.        -4.        -4.        -4.       ]
 [ 9.096904   7.6039405 11.706295   8.996958 ]]
[[ 0.  1.  2.  3.]
 [ 4.  5.  6.  7.]
 [ 8.  9. 10. 11.]]


In [12]:
t2 = add_reverse